In [20]:
import os
import json
import sys
import pandas as pd
from dataset import Drive360Loader
from torchvision import models
import torch.nn as nn
import torch
import numpy as np
from scipy import interpolate
import torch.optim as optim
import datetime
from itertools import islice
import torch
import torch.nn as nn
from sklearn.metrics import mean_squared_error
import pretrainedmodels
from scipy.signal import savgol_filter

In [21]:
save_dir = "./"
os.chdir(save_dir)
run_val = True #run a validation at the end or not
detail_size = 200 #How many batches to print results to console
output_states = list(range(0, 100)) #which epochs to save the submission file for
max_epoch = 5 #number of epochs to run
min_start_batch_idx = -1 #if restarting, what batch to restart running at, -1 means from the beginning
extrapolate = True #whether to extrapolate the data. If True but it is trained on the full dataset, reverts to False anyway
full_start = 100 # Each chapter in the submission file starts at 10s
run_time = datetime.datetime.now()

np.random.seed(88)
torch.manual_seed(88)

## Loading data from Drive360 dataset.
# load the config.json file that specifies data 
# location parameters and other hyperparameters 
# required.
config = json.load(open(save_dir + 'evaluate_config_1.json'))

In [22]:
# load training data
df = pd.read_csv('./Drive360Images_160_90/drive360challenge_train-split_train.csv', usecols=['canSteering', 'canSpeed'])
steer_min = df['canSteering'].min()
steer_max = df['canSteering'].max()
speed_min = df['canSpeed'].min()
speed_max = df['canSpeed'].max()

steer_counts, steer_edges = np.histogramdd(df['canSteering'].to_numpy(), bins=100)
speed_counts, speed_edges = np.histogramdd(df['canSpeed'].to_numpy(), bins=30) 
steer_edges = steer_edges[0]
speed_edges = speed_edges[0]

In [23]:

# Window should be the max odd number <= num_data_points and cannot exceed max_window
def get_window(num_data_points, max_window):
    assert max_window % 2 == 1
    return min(max_window, (num_data_points - 1) // 2 * 2 + 1)

def knn_online_smooth(speed):
    smoothed = np.zeros(len(speed))
    for i in range(len(smoothed)):
        num_data_points = i + 1
        max_window = 5
        window = get_window(num_data_points, max_window)
        future_window = window // 2
        past_window = window - future_window
        start = max(0, num_data_points - past_window)
        smoothed[i] = speed[i]
        history = speed[start:num_data_points]
        for _ in range(future_window):
            history.append(history[-1])
        smoothed[i] = np.mean(history)
    return smoothed


import matplotlib.pyplot as plt
def plot(ensemble_submission):
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot(ensemble_submission['canSpeed'][700:800], 'r.', label= 'Unsmoothed')
    ax.plot(knn_online_smooth(ensemble_submission['canSpeed'])[700:800], 'g--', label="kNN smoothed")
    ax.plot(savgol_filter(ensemble_submission['canSpeed'], 51, 2)[700:800], 'b--', label="savgol filter (ideal)")
    plt.legend(loc='best')

In [24]:
test_loader = Drive360Loader(config, 'test')
sample_indices = test_loader.drive360.indices 

test_df = pd.read_csv('./Drive360Images_160_90/drive360challenge_validation.csv', usecols=['canSteering', 'canSpeed'])
test_df_sampled = test_df.iloc[sample_indices,:]
actual_steering  = test_df_sampled['canSteering'].values
actual_speed = test_df_sampled['canSpeed'].values

Phase: test # of data: 10302


In [25]:
#picking models for ensembling, for this we look at the validation errors of individual models and pick the epoch with the best performance
#next models are sorted according to mse and we ensemble the top-k models by varying k
model_keys = [
    'resnet34',  # original
    'resnext101_32x4d',
    'se_resnet50',
    'se_resnet101',
    'se_resnet152',
    'se_resnext50_32x4d',
    'se_resnext101_32x4d',
    'densenet121',
    'densenet161',
    'densenet169',
    'densenet201',
    'resnet50_swsl',
    'resnext50_32x4d_swsl',
    'resnext101_32x4d_swsl',
    'resnet50_ssl',
    'resnext50_32x4d_ssl',
    'resnext101_32x4d_ssl'
]

prefix = '2020-03-26 18h48m15s'
results = []
for model_key in model_keys:
    tmp = []
    one_val = 0
    two_val = 0
    one = []
    two = []
    if os.path.isfile('./evaluate_model_1/val_results 2020-03-26 18h48m15s.json'):
        with open('./evaluate_model_1/val_results 2020-03-26 18h48m15s.json') as f:
            data = json.load(f)
            data = data[model_key]
            one_val = data['mse_steering']
            one = [model_key, 1, data['mse_steering'], data['mse_speed'], data['mse_combined']]

    if os.path.isfile('./evaluate_model_1/val_results {} after_epoch 2 {}.json'.format(model_key, prefix)):
        with open('./evaluate_model_1/val_results {} after_epoch 2 {}.json'.format(model_key, prefix)) as f:
            data = json.load(f)
            two_val = data['mse_steering']
            two = [model_key, 2, data['mse_steering'], data['mse_speed'], data['mse_combined']]
    
    if one_val < two_val:
        results.append(one)
    else:
        results.append(two)

df = pd.DataFrame(results)
df = df.sort_values(by=[2])
df

,0,1,2,3,4
2,se_resnet50,2,777.687256,4.924152,782.611389
12,resnext50_32x4d_swsl,2,791.485046,4.953746,796.438782
0,resnet34,2,806.442871,5.194991,811.637878
6,se_resnext101_32x4d,2,841.470825,5.055166,846.526001
10,densenet201,1,850.278687,5.738891,856.017578
11,resnet50_swsl,1,869.341492,5.568923,874.910400
5,se_resnext50_32x4d,1,870.928223,5.177955,876.106201
13,resnext101_32x4d_swsl,1,872.970154,5.258327,878.228455
7,densenet121,2,893.630432,4.537238,898.167664
14,resnet50_ssl,2,905.240540,4.825393,910.065918


In [26]:
#fetching mse on test data
test_loader = Drive360Loader(config, 'test')
sample_indices = test_loader.drive360.indices 

test_df = pd.read_csv('./Drive360Images_160_90/drive360challenge_validation.csv', usecols=['canSteering', 'canSpeed'])
test_df_sampled = test_df.iloc[sample_indices,:]
steering  = test_df_sampled['canSteering'].values
speed = test_df_sampled['canSpeed'].values

#individual models taking part in one of the two ensembles
model_keys = [
    'resnet34',  # original
    'resnext101_32x4d',
    'se_resnet50',
    'densenet121',
    'resnext50_32x4d_swsl',
    'resnext101_32x4d_swsl',
    'resnet50_ssl',
]
prefix = '2020-03-26 18h48m15s'
results = []
for model_key in model_keys:
    tmp = []
    if os.path.isfile('./evaluate_model_1/{}_{}_Submission.csv'.format(prefix, model_key)):
        submission_df = pd.read_csv('./evaluate_model_1/{}_{}_Submission.csv'.format(prefix, model_key), usecols=['canSteering', 'canSpeed'])
        submission_steering  = submission_df['canSteering'].values
        submission_speed = submission_df['canSpeed'].values
        steering_loss = mean_squared_error(steering, submission_steering)
        speed_loss = mean_squared_error(speed, submission_speed)

        results.append([model_key, steering_loss, speed_loss, steering_loss+speed_loss])
    
    if os.path.isfile('./evaluate_model_1/{}_{}_Submission after_epoch 2.csv'.format(prefix, model_key)):
        submission_df = pd.read_csv('./evaluate_model_1/{}_{}_Submission after_epoch 2.csv'.format(prefix, model_key), usecols=['canSteering', 'canSpeed'])
        submission_steering  = submission_df['canSteering'].values
        submission_speed = submission_df['canSpeed'].values
        steering_loss = mean_squared_error(steering, submission_steering)
        speed_loss = mean_squared_error(speed, submission_speed)

        results.append([model_key, steering_loss, speed_loss, steering_loss+speed_loss])

df = pd.DataFrame(results)
df

Phase: test # of data: 10302


,0,1,2,3
0,resnet34,1128.443374,6.877611,1135.320985
1,resnet34,844.734238,6.302661,851.036899
2,resnext101_32x4d,1131.427659,7.496218,1138.923878
3,resnext101_32x4d,1023.681785,6.777642,1030.459427
4,se_resnet50,1043.040850,7.041320,1050.082170
5,se_resnet50,1010.806740,6.407365,1017.214105
6,densenet121,896.799353,7.846518,904.645871
7,densenet121,1006.132487,7.235354,1013.367842
8,resnext50_32x4d_swsl,1104.444422,7.119119,1111.563541
9,resnext50_32x4d_swsl,950.250814,6.973341,957.224155


In [27]:
# load submission files
num_rows = 10302
ensemble_submission = {
    'canSteering': [], 
    'canSpeed': []
}

# ensemble angle
# timestamp of creation
prefix = '2020-03-26 18h48m15s'

#best models sorted according to validation mse
steer_submission_filenames = [
    'se_resnet50_Submission after_epoch 2',
    'resnext50_32x4d_swsl_Submission after_epoch 2',
    'resnet34_Submission after_epoch 2',
    'se_resnext101_32x4d_Submission after_epoch 2',
    'densenet201_Submission',
    'resnet50_swsl_Submission',
    'se_resnext50_32x4d_Submission',
    'resnext101_32x4d_swsl_Submission',
    'densenet121_Submission after_epoch 2',
    'resnet50_ssl_Submission after_epoch 2',
    'densenet161_Submission after_epoch 2',
    'se_resnet101_Submission after_epoch 2',
    'resnext50_32x4d_ssl_Submission after_epoch 2'
]

#evaluating the ensemble for 'speed' alone
print('Evaluating an unweighted average over top-k models')
for num_models_to_include in range(1, len(steer_submission_filenames)+1):
    ensemble_submission['canSteering'] = []
    files_to_include = steer_submission_filenames[0:num_models_to_include]
    num_steer_submissions = len(files_to_include)
    steer_submissions = np.zeros((num_steer_submissions, num_rows))

    for i in range(num_steer_submissions):
        df_submission = pd.read_csv('./evaluate_model_1/{}_{}.csv'.format(prefix, steer_submission_filenames[i]), usecols=['canSteering'])
        assert len(df_submission)== num_rows
        values = df_submission.to_numpy().squeeze()
        steer_submissions[i] = values
    
    ensemble_submission['canSteering'] = np.sum(steer_submissions, axis=0)/num_steer_submissions
    

    submission_steering  = ensemble_submission['canSteering']
    steering_loss = mean_squared_error(actual_steering, submission_steering)
    print('{} : {}'.format(num_models_to_include, steering_loss))
    
print('Evaluating a weighted average over top-k models')
for num_models_to_include in range(1, len(steer_submission_filenames)+1):
    ensemble_submission['canSteering'] = []
    files_to_include = steer_submission_filenames[0:num_models_to_include]
    num_steer_submissions = len(files_to_include)
    steer_submissions = np.zeros((num_steer_submissions, num_rows))
    submission_steer_counts = np.zeros((num_steer_submissions, num_rows))

    for i in range(num_steer_submissions):
        df_submission = pd.read_csv('./evaluate_model_1/{}_{}.csv'.format(prefix, steer_submission_filenames[i]), usecols=['canSteering'])
        assert len(df_submission)== num_rows
        values = df_submission.to_numpy().squeeze()

        steer_submissions[i] = values
        values = np.clip(values, steer_min + 1e-10, steer_max - 1e-10)
        indices = np.zeros(values.shape)
        indices = np.digitize(values, steer_edges) - 1
        indices = indices.astype(int)
        for j in range(num_rows):
            submission_steer_counts[i, j] = steer_counts[indices[j]]
    
    # select the most likely submission for each row
    for j in range(num_rows):
        steer_counts_j = submission_steer_counts[:, j]
        steer_j = steer_submissions[:, j]
        canSteering = np.dot(steer_counts_j, steer_j) / steer_counts_j.sum()
        ensemble_submission['canSteering'].append(canSteering) 

    knn_online_smooth(ensemble_submission['canSteering'])    
    #evaluate current ensemble
    submission_steering  = ensemble_submission['canSteering']
    steering_loss = mean_squared_error(actual_steering, submission_steering)
    print('{} : {}'.format(num_models_to_include, steering_loss))

    

Evaluating an unweighted average over top-k models
1 : 1010.8067402460841
2 : 911.369586652079
3 : 859.3858075812911
4 : 874.3605168292646
5 : 862.336051202201
6 : 878.1510106423065
7 : 876.5107442848979
8 : 882.4896490298063
9 : 884.9976223946859
10 : 882.5699239698016
11 : 867.3415096707383
12 : 874.2465085178909
13 : 872.9509404172054
Evaluating a weighted average over top-k models
1 : 1010.8067402460841
2 : 966.8864848236464
3 : 948.2559965533372
4 : 962.5142827600064
5 : 968.870507966195
6 : 980.041506348763
7 : 972.557019741362
8 : 987.6645133839459
9 : 986.2847554409601
10 : 1004.7693420251735
11 : 1000.1205469167606
12 : 1017.3486494850639
13 : 1014.583457172976


In [28]:
# ensemble speed

#best models sorted according to validation mse
speed_submission_filenames = ['densenet121_Submission after_epoch 2',
                             'resnet50_ssl_Submission after_epoch 2',
                             'se_resnet50_Submission after_epoch 2',
                              'resnext50_32x4d_swsl_Submission after_epoch 2',
                             'se_resnext101_32x4d_Submission after_epoch 2']

print('Evaluating an unweighted average over top-k models')
for num_models_to_include in range(1, len(speed_submission_filenames)+1):
    ensemble_submission['canSpeed'] = []
    files_to_include = speed_submission_filenames[0:num_models_to_include]
    num_speed_submissions = len(files_to_include)
    speed_submissions = np.zeros((num_speed_submissions, num_rows))
    
    for i in range(num_speed_submissions):
        df_submission = pd.read_csv('./evaluate_model_1/{}_{}.csv'.format(prefix, speed_submission_filenames[i]), usecols=['canSpeed'])
        assert len(df_submission)== num_rows
        values = df_submission.to_numpy().squeeze()
        speed_submissions[i] = values
        
    ensemble_submission['canSpeed'] = np.sum(speed_submissions, axis=0)/num_speed_submissions    
    submission_speed = ensemble_submission['canSpeed']
    speed_loss = mean_squared_error(actual_speed, submission_speed)
    print('{} : {}'.format(num_models_to_include, speed_loss))
    
    
print('Evaluating a weighted average over top-k models')
for num_models_to_include in range(1, len(speed_submission_filenames)+1):
    ensemble_submission['canSpeed'] = []
    files_to_include = speed_submission_filenames[0:num_models_to_include]
    num_speed_submissions = len(files_to_include)
    speed_submissions = np.zeros((num_speed_submissions, num_rows))
    submission_speed_counts = np.zeros((num_speed_submissions, num_rows))

    for i in range(num_speed_submissions):
        df_submission = pd.read_csv('./evaluate_model_1/{}_{}.csv'.format(prefix, speed_submission_filenames[i]), usecols=['canSpeed'])
        assert len(df_submission)== num_rows
        values = df_submission.to_numpy().squeeze()
        speed_submissions[i] = values
        values = np.clip(values, speed_min + 1e-10, speed_max - 1e-10)
        indices = np.zeros(values.shape)
        indices = np.digitize(values, speed_edges) - 1
        indices = indices.astype(int)
        for j in range(num_rows):
            submission_speed_counts[i, j] = speed_counts[indices[j]]


    # select the most likely submission for each row
    for j in range(num_rows):
        speed_counts_j = submission_speed_counts[:, j]
        speed_j = speed_submissions[:, j]
        canSpeed = np.dot(speed_counts_j, speed_j) / speed_counts_j.sum()
        ensemble_submission['canSpeed'].append(canSpeed)
    
    knn_online_smooth(ensemble_submission['canSpeed'])
    submission_speed = ensemble_submission['canSpeed']
    speed_loss = mean_squared_error(actual_speed, submission_speed)
    print('{} : {}'.format(num_models_to_include, speed_loss))

Evaluating an unweighted average over top-k models
1 : 7.235354284069027
2 : 6.2666308972710505
3 : 5.887102216896609
4 : 5.808974328037417
5 : 5.91400310606215
Evaluating a weighted average over top-k models
1 : 7.235354284069027
2 : 6.231772729706765
3 : 5.84257917927658
4 : 5.731726722464196
5 : 5.814884517477352
